In [4]:
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import time
from datetime import datetime
lm=WordNetLemmatizer()
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout

In [5]:

df=pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
comments=df['review'].to_list()

In [7]:
corpus=[]
start=datetime.now()
print(f'process started {start}')
for comment in comments:
    review=re.sub('[^a-zA-Z]',' ',comment)
    review=review.lower()
    review=review.split()
    review=[lm.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review=' '.join(review)
    corpus.append(review)
end=datetime.now()
print(f' time taken to complete:- {end-start}')


process started 2024-08-08 12:06:59.513513
 time taken to complete:- 0:44:11.101830


In [8]:
corpus[0]

'one reviewer mentioned watching oz episode hooked right exactly happened br br first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use word br br called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home many aryan muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far away br br would say main appeal show due fact go show dare forget pretty picture painted mainstream audience forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due lack street skill prison experi

In [9]:
df['review']=corpus

In [15]:
df.to_csv('Imdb.csv',index=False)

In [10]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [11]:
length=[ len(sent.split()) for sent in corpus]
print(sum(length)/len(length))
print(max(length))
print(length.index(max(length)))
corpus[31481],max(length)


122.19208
1442
31481


('match tag team table match bubba ray spike dudley v eddie guerrero chris benoit bubba ray spike dudley started thing tag team table match eddie guerrero chris benoit according rule match opponent go table order get win benoit guerrero heated early taking turn hammering first spike bubba ray german suplex benoit bubba took wind dudley brother spike tried help brother referee restrained benoit guerrero ganged corner benoit stomping away bubba guerrero set table outside spike dashed ring somersaulted top rope onto guerrero outside recovering taking care spike guerrero slipped table ring helped wolverine set tandem set double superplex middle rope would put bubba table spike knocked table right brother came crashing guerrero benoit propped another table corner tried irish whip spike bubba dashed blocked brother bubba caught fire lifted opponent back body drop bubba slammed guerrero spike stomped wolverine top rope bubba held benoit bay spike soar wassup headbutt shortly benoit latched sp

In [12]:
voc_size=20000
onehot_repr=[one_hot(words,voc_size) for words in corpus]
# print(onehot_repr)


In [17]:
onehot_repr[0]

[4365,
 8533,
 13837,
 4328,
 15648,
 1320,
 12427,
 15320,
 11775,
 7744,
 14299,
 14299,
 9461,
 2097,
 3035,
 15648,
 18682,
 4493,
 11705,
 231,
 2855,
 15320,
 11935,
 8013,
 12876,
 17382,
 134,
 4321,
 7397,
 17382,
 18564,
 18279,
 18847,
 5027,
 5559,
 231,
 17325,
 9572,
 4621,
 11935,
 14299,
 14299,
 18149,
 15648,
 9972,
 8963,
 19083,
 16975,
 16811,
 19317,
 2662,
 10311,
 11647,
 6785,
 16936,
 12263,
 4040,
 12438,
 8001,
 16119,
 17520,
 9338,
 2582,
 5303,
 17284,
 14954,
 1693,
 16936,
 13742,
 18417,
 3540,
 19186,
 12684,
 2188,
 10240,
 15243,
 13462,
 13926,
 15555,
 9718,
 5591,
 12815,
 15279,
 3648,
 7409,
 8287,
 2985,
 14299,
 14299,
 14434,
 17812,
 17779,
 3594,
 17382,
 844,
 8564,
 8013,
 17382,
 4681,
 10396,
 7796,
 9980,
 4368,
 17456,
 11676,
 10396,
 3384,
 10396,
 3856,
 15648,
 2203,
 17679,
 9461,
 1320,
 11537,
 16679,
 3035,
 11480,
 10782,
 17812,
 11556,
 4415,
 17842,
 10342,
 15648,
 6213,
 4627,
 17284,
 18871,
 12054,
 231,
 231,
 8269,


In [18]:
sent_length=30
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [19]:
embedded_docs[0]

array([ 1992, 11753,  7958,  2985, 18958,  3942, 19249, 11431, 14926,
         706, 12438,  6804,   844,  9306, 12908,  5907, 12438, 16580,
        4328, 15648,  2490, 10546,  4698, 13473, 15879,  7029,  7958,
        7309, 10269, 17795])

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Bidirectional,LSTM,Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [21]:
# Adjusting output dimensions in your model
model = Sequential()
model.add(Embedding(input_dim=50000, output_dim=30))  # Reduced embedding output dimension
model.add(Bidirectional(LSTM(100,dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','MSE'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [22]:
df['sentiment']=df['sentiment'].map({'positive':1,'negative':0})

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(embedded_docs,df['sentiment'], test_size=0.2, random_state=42,stratify=df['sentiment'])


In [24]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((40000, 30), (40000,), (10000, 30), (10000,))

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42,stratify=y_train)

In [26]:
X_train.shape,X_val.shape,y_train.shape,y_val.shape

((32000, 30), (8000, 30), (32000,), (8000,))

In [27]:
from datetime import datetime
s=datetime.now()
print(f'process started {s}')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history=model.fit(X_train, y_train, epochs=100, batch_size=50,validation_data=(X_val, y_val), callbacks=[early_stopping])
e=datetime.now()
print(f'time taken:- {e-s}')

process started 2024-08-08 12:53:02.695890
Epoch 1/100
640/640 ━━━━━━━━━━━━━━━━━━━━ 23s 30ms/step - MSE: 0.1858 - accuracy: 0.6984 - loss: 0.5466 - val_MSE: 0.1373 - val_accuracy: 0.8023 - val_loss: 0.4254
Epoch 2/100
640/640 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - MSE: 0.1007 - accuracy: 0.8638 - loss: 0.3313 - val_MSE: 0.1379 - val_accuracy: 0.8008 - val_loss: 0.4280
Epoch 3/100
640/640 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - MSE: 0.0779 - accuracy: 0.8966 - loss: 0.2645 - val_MSE: 0.1550 - val_accuracy: 0.7905 - val_loss: 0.5022
Epoch 4/100
640/640 ━━━━━━━━━━━━━━━━━━━━ 18s 29ms/step - MSE: 0.0551 - accuracy: 0.9297 - loss: 0.1962 - val_MSE: 0.1617 - val_accuracy: 0.7853 - val_loss: 0.5325
Epoch 5/100
640/640 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - MSE: 0.0413 - accuracy: 0.9479 - loss: 0.1509 - val_MSE: 0.1730 - val_accuracy: 0.7810 - val_loss: 0.6298
Epoch 6/100
640/640 ━━━━━━━━━━━━━━━━━━━━ 20s 32ms/step - MSE: 0.0290 - accuracy: 0.9615 - loss: 0.1066 - val_MSE: 0.1838 - val_accuracy: 0.775

In [28]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (50, 30, 30)           │     1,500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (50, 200)              │       104,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (50, 64)               │        12,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (50, 64)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (50, 1)                │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,853,189 (18.51 MB)

 Trainable params: 1,617,729 (6.17 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,235,460 (12.34 MB)

In [29]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred>0.5,1,0)

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step


In [33]:
from sklearn.metrics import  accuracy_score,confusion_matrix
print(accuracy_score(y_pred,y_test))
confusion_matrix(y_pred,y_test)

0.8059


array([[4328, 1269],
       [ 672, 3731]], dtype=int64)

In [34]:
accuracy_score(y_test,y_pred)

0.8059

In [35]:
model.save('MovieSentiment.keras')

In [37]:
def predi(corpus):
  # n=int(input('enter number of reviews'))
  # corpus=[]
  # for i in range(n):
  #       movie=input('enter sentiment:- ')
  #       corpus.append(movie)
  for movie in corpus:
      review=re.sub('[^a-zA-Z]',' ',movie)
      review=review.lower()
      review=review.split()
      review=[lm.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
      review=' '.join(review)
      print(review)
      review=one_hot(review,25000)
      review=pad_sequences([review],padding='pre',maxlen=30)
      output=model.predict(review)
      score=output
      output=np.where(output>0.5,1,0)
      if output==1:
        print(f'{score} :- positive')
      else:
        print(f'{score} :- negative')



In [38]:
predi([
    "The most amazing film I have ever seen. The acting is superb and the story is truly heartwarming. A must watch.",
    "I absolutely loved this movie. It was so well made and the characters were incredibly engaging. Highly recommend.",
    "Fantastic movie! Great plot and excellent performances by the cast. One of the best movies of the year.",
    "A beautifully shot film with a compelling story and brilliant acting. Definitely worth watching.",
    "This movie was a delight from start to finish. I was completely captivated by the story and the characters."
])

amazing film ever seen acting superb story truly heartwarming must watch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
[[0.61931276]] :- positive
absolutely loved movie well made character incredibly engaging highly recommend
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[[0.62214017]] :- positive
fantastic movie great plot excellent performance cast one best movie year
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[[0.6986837]] :- positive
beautifully shot film compelling story brilliant acting definitely worth watching
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[[0.9036684]] :- positive
movie delight start finish completely captivated story character
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
[[0.68266505]] :- positive
